In [86]:
import tensorflow as tf
import larq as lq
import numpy as np
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
import tensorflow.keras as keras
class Multibin(tf.keras.layers.Layer):

    def __init__(self, out_channel, kernel_size, M=1, **kwargs):
        self.out_channel = out_channel
        self.kernel_size = kernel_size
        self.M=M
        self.kwargs=kwargs
        super(Multibin, self).__init__(**kwargs)


    def build(self, input_shape):
#         super(Multibin, self).build(**kwargs)
        # Create a trainable weight variable for this layer.
        self.layers=[]
        for i in range(self.M):
            self.layers.append(lq.layers.QuantConv2D(self.out_channel, 
                                                  self.kernel_size, 
                                                  padding="same", 
                                                  **self.kwargs))
        # Be sure to call this at the end
    def conv_output_length(input_length, filter_size,
                       padding, stride, dilation=1):
        """Determines output length of a convolution given input length.
        # Arguments
            input_length: integer.
            filter_size: integer.
            padding: one of `"same"`, `"valid"`, `"full"`.
            stride: integer.
            dilation: dilation rate, integer.
        # Returns
            The output length (integer).
        """
        if input_length is None:
            return None
        assert padding in {'same', 'valid', 'full', 'causal'}
        dilated_filter_size = (filter_size - 1) * dilation + 1
        if padding == 'same':
            output_length = input_length
        elif padding == 'valid':
            output_length = input_length - dilated_filter_size + 1
        elif padding == 'causal':
            output_length = input_length
        elif padding == 'full':
            output_length = input_length + dilated_filter_size - 1
        return (output_length + stride - 1) // stride
    
    def compute_output_shape(self, input_shape):
        if self.data_format == 'channels_last':
            space = input_shape[1:-1]
        elif self.data_format == 'channels_first':
            space = input_shape[2:]
        new_space = []
        for i in range(len(space)):
            new_dim = conv_output_length(
                space[i],
                self.kernel_size[i],
                padding=self.padding,
                stride=self.strides[i],
                dilation=self.dilation_rate[i])
            new_space.append(new_dim)
        if self.data_format == 'channels_last':
            return (input_shape[0],) + tuple(new_space) + (self.filters,)
        elif self.data_format == 'channels_first':
            return (input_shape[0], self.filters) + tuple(new_space)


    def call(self, inputs):
        outputs=self.layers[0](inputs)
        for i in range(1, self.M):
            outputs = outputs+self.layers[i](inputs)
        return outputs

    # def compute_output_shape(self, input_shape):
    #   shape = tf.TensorShape(input_shape).as_list()
    #   shape[-1] = self.out_channel
    #   return tf.TensorShape(shape)

    def get_config(self):
        base_config = super(Multibin, self).get_config()
        base_config['output_dim'] = self.output_dim

    @classmethod
    def from_config(cls, config):
        return cls(**config)


In [87]:
# dataset
data = np.random.random((1000, 32 , 32, 3))
labels = np.random.random((1000, 10))
# Create a model using the custom layer
inputs = tf.keras.Input(shape=(32,32,3))
Multibin1 = Multibin(10,3, name="MultiBin_1")
out1 = Multibin1(inputs)
out2 = tf.keras.layers.Dense(10)(out1)
final = tf.keras.layers.Activation('softmax')(out2)

model1=Model(inputs=inputs, outputs=final)


# model = Sequential()
# model.add(Multibin1)
# model.add(tf.keras.layers.Dense(10, name="final_out"))
# model.add(tf.keras.layers.Activation('softmax', name="softmax"))


In [88]:
model2_Multibin1 = Multibin(10,3, name="MultiBin_1")
model2_out1 = Multibin1(inputs)
model2_out2 = tf.keras.layers.Dense(10)(model2_out1)
model2_final = tf.keras.layers.Activation('softmax')(model2_out2)
model2 = Model(inputs=inputs,outputs=model2_final)


In [89]:
tealoss=K.sum(out1-model2_out1,axis=[1,2,3]) + K.sum(out2-model2_out1,axis=[1,2,3])

In [90]:
print(tealoss)

Tensor("add_7:0", shape=(?,), dtype=float32)


In [91]:
model.compile(optimizer=tf.train.RMSPropOptimizer(0.001),
              loss=[tealoss], metrics=['accuracy'])

model.fit(data, [labels, modelB.predict(data)], epochs=10, batch_size=32)

ValueError: ('Could not interpret loss function identifier:', <tf.Tensor 'add_7:0' shape=(?,) dtype=float32>)

In [22]:
print(final)

Tensor("activation/truediv:0", shape=(?, 32, 32, 10), dtype=float32)


In [30]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

a = Input(shape=(32,))
b = Dense(32)(a)
model = Model(inputs=a, outputs=b)

In [32]:
print(inputs)

Tensor("input_15:0", shape=(?, 32, 32, 3), dtype=float32)


In [ ]:
def at(x):
    out = F.normalize(x.pow(2).mean(1).view(x.size(0), -1))
    # print("out is ", out)
    return out

def at_loss(x, y):
    return (at(x) - at(y)).pow(2).mean()